In [1]:
from itertools import count, cycle

def Input(day):
    "Open this day's input file."
    filename = 'input{}.txt'.format(day)
    try:
        return open(filename)
    except FileNotFoundError:
        print("No input file found for day", day)

# [Day 1](http://adventofcode.com/2017/day/1): Inverse Captcha
### Part one
Find the sum of all digits that match the next digit in the list. The list is circular, so the digit after the last digit is the first digit in the list.

In [2]:
def parse(text):
    "Parse a string of numbers into a list of digits"
    return [int(n) for n in list(text.strip())]

def captcha_n(digits, k):
    sum = 0
    
    for i, n in enumerate(digits):
        if n == digits[(i + k) % len(digits)]:
            sum += n
    
    return sum

def captcha(digits):
    return captcha_n(digits, 1)

assert parse("1234") == [1, 2, 3, 4]
assert captcha([1, 1, 2, 2]) == 3
assert captcha([1, 1, 1, 1]) == 4
assert captcha([1, 2, 3, 4]) == 0
assert captcha([9, 1, 2, 1, 2, 1, 2, 9]) == 9

captcha(parse(Input(1).read()))

1034

#### Part two
Now, instead of considering the next digit, it wants you to consider the digit halfway around the circular list.

In [3]:
def captcha_halfway(digits):
    assert len(digits) % 2 == 0
    return captcha_n(digits, len(digits) // 2)

assert captcha_halfway([1, 2, 1, 2]) == 6
assert captcha_halfway([1, 2, 3, 4, 2, 5]) == 4
assert captcha_halfway([1, 2, 3, 1, 2, 3]) == 12
assert captcha_halfway([1, 2, 1, 3, 1, 4, 1, 5]) == 4

captcha_halfway(parse(Input(1).read()))

1356

# [Day 2](http://adventofcode.com/2017/day/2): Corruption Checksum
### Part one
For each row, determine the difference between the largest value and the smallest value; the checksum is the sum of all of these differences.

In [4]:
def max_range(row):
    low = float("inf")
    high = 0
    
    for i in row:
        if i < low:
            low = i
        if i > high:
            high = i
    
    return high - low

def checksum(file, row_func):
    sum = 0
    for row in file:
        sum += row_func(row)
        
    return sum

def parse(text):
    "Parse lines of tab separated numbers into lists of integers"
    res = []
    for line in text.strip().split("\n"):
        res.append([int(n) for n in line.split("\t")])
    return res

assert parse("1236	741	557	1029	144") == [[1236, 741, 557, 1029, 144]]
assert checksum_row([5, 1, 9, 5]) == 8
test_input = [[5, 1, 9, 5], [7, 5, 3], [2, 4, 6, 8]]
assert checksum(test_input, max_range) == 18

checksum(parse(Input(2).read()), max_range)

NameError: name 'checksum_row' is not defined

### Part two
It sounds like the goal is to find the only two numbers in each row where one evenly divides the other - that is, where the result of the division operation is a whole number. They would like you to find those numbers on each line, divide them, and add up each line's result.

In [ ]:
def quotient(row):
    row.sort()
    for i, n in enumerate(row):
        for m in row[:i:-1]:
            if m % n == 0:
                return m // n

assert checksum_row_divisor([5, 9, 2, 8]) == 4

checksum(parse(Input(2).read()), quotient)

# [Day 3](http://adventofcode.com/2017/day/3): Spiral Memory
### Part one

You come across an experimental new kind of memory stored on an infinite two-dimensional grid.

Each square on the grid is allocated in a spiral pattern starting at a location marked 1 and then counting up while spiraling outward. For example, the first few squares are allocated like this:

```
17  16  15  14  13
18   5   4   3  12
19   6   1   2  11
20   7   8   9  10
21  22  23---> ...
```
While this is very space-efficient (no squares are skipped), requested data must be carried back to square 1 (the location of the only access port for this memory system) by programs that can only move up, down, left, or right. They always take the shortest path: the Manhattan Distance between the location of the data and square 1.

For example:

Data from square 1 is carried 0 steps, since it's at the access port.
Data from square 12 is carried 3 steps, such as: down, left, left.
Data from square 23 is carried only 2 steps: up twice.
Data from square 1024 must be carried 31 steps.
How many steps are required to carry the data from the square identified in your puzzle input all the way to the access port?

In [ ]:
def center_distance(n):
    "Given a number, return its distance from the center of the spiral map"
    box_height = 1
    x = 0
    
    # Find box height
    while x < n:
        box_height += 2
        x = box_height * box_height
    
    # Find nearest corner
    corner = x
    while corner > n:
        corner -= box_height - 1
    
    if n - corner > corner + (box_height - 1) - n:
        corner = corner + (box_height - 1)
    
    # Distance from center to nearest corner - distance from nearest corner to point
    diag = (box_height // 2) * 2
    print(diag - abs(corner - n))

spiral_coords(265149)


### Part 2
As a stress test on the system, the programs here clear the grid and then store the value 1 in square 1. Then, in the same allocation order as shown above, they store the sum of the values in all adjacent squares, including diagonals.

So, the first few squares' values are chosen as follows:

Square 1 starts with the value 1.
Square 2 has only one adjacent filled square (with value 1), so it also stores 1.
Square 3 has both of the above squares as neighbors and stores the sum of their values, 2.
Square 4 has all three of the aforementioned squares as neighbors and stores the sum of their values, 4.
Square 5 only has the first and fourth squares as neighbors, so it gets the value 5.
Once a square is written, its value does not change. Therefore, the first few squares would receive the following values:

```
147  142  133  122   59
304    5    4    2   57
330   10    1    1   54
351   11   23   25   26
362  747  806--->   ...
```
What is the first value written that is larger than your puzzle input?



In [ ]:
class SpiralLocation:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.value = 1

def get_neighbors(x, y):
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
            (x+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))
        
def spiral(end, cell_func):
    "Build a spiraled map from 1..end, setting the value at each cell with cell_func"
    loc = SpiralLocation()
    width = 1   
    spiral = {(loc.x, loc.y): loc.value}

    down = lambda x: x - 1
    up = lambda x: x + 1
    
    def spiral_side(axis, direction):
        corner = max_coord if direction == up else -max_coord
        current = getattr(loc, axis)

        while current != corner:
            setattr(loc, axis, direction(current))
            loc.value = cell_func(loc, spiral)
            spiral[(loc.x, loc.y)] = loc.value
            if loc.value >= end:
                return True
            current = getattr(loc, axis)
        return False
        
    while loc.value < end:
        width += 2
        max_coord = width // 2
        
        if spiral_side("x", up):
            return spiral
        if spiral_side("y", up):
            return spiral
        if spiral_side("x", down):
            return spiral
        if spiral_side("y", down):
            return spiral
        if spiral_side("x", up):
            return spiral
            
    return spiral

def neighbor_sum(loc, spiral):
    res = 0
    neighbor_coords = get_neighbors(loc.x, loc.y)
    for n in neighbor_coords:
        value = spiral.get(n)
        if value:
            res += value

    return res
 
res = spiral(25, lambda loc, spiral: loc.value + 1)
assert res[(0, 2)] == 15
assert res[(0, 0)] == 1
assert res[(2, -2)] == 25
assert res[(-1, 0)] == 6

test_spiral = {(0, 1): 4, (-1, 1): 5, (0, 0): 1, (-1, 0): 6, (-1, -1): 7, (0, -1): 8, (1, 0): 2, (1, -1): 9, (1, 1): 3}
test_loc = SpiralLocation()
test_loc.x = -1
test_loc.y = 0
assert neighbor_sum(test_loc, test_spiral) == 25

neighbor_spiral = spiral(750, neighbor_sum)
assert sorted(list(neighbor_spiral.values()))[-1] == 806

neighbor_spiral = spiral(265149, neighbor_sum)
sorted(list(neighbor_spiral.values()))[-1]

# [Day 4](http://adventofcode.com/2017/day/4): High-Entropy Passphrases

### Part one
A new system policy has been put in place that requires all accounts to use a passphrase instead of simply a password. A passphrase consists of a series of words (lowercase letters) separated by spaces.

To ensure security, a valid passphrase must contain no duplicate words.

For example:
```
aa bb cc dd ee is valid.
aa bb cc dd aa is not valid - the word aa appears more than once.
aa bb cc dd aaa is valid - aa and aaa count as different words.
```
The system's full passphrase list is available as your puzzle input. How many passphrases are valid?

In [ ]:
def no_dupes(words):
    return len(words) - len(set(words)) == 0

def parse_words(text):
    return text.split(" ")

def valid_passphrase_count(lines, filter_func):
    parsed_lines = map(parse_words, lines)
    return len(list(filter(filter_func, parsed_lines)))


assert no_dupes(["aa", "bb", "cc", "dd", "ee"]) == True
assert no_dupes(["aa", "bb", "cc", "dd", "aa"]) == False
assert no_dupes(["aa", "bb", "cc", "dd", "aaa"]) == True
assert parse_words("aa bb cc") == ["aa", "bb", "cc"]

valid_passphrase_count(Input(4).read().splitlines(), no_dupes)

### Part two
For added security, yet another system policy has been put in place. Now, a valid passphrase must contain no two words that are anagrams of each other - that is, a passphrase is invalid if any word's letters can be rearranged to form any other word in the passphrase.

For example:
```
abcde fghij is a valid passphrase.
abcde xyz ecdab is not valid - the letters from the third word can be rearranged to form the first word.
a ab abc abd abf abj is a valid passphrase, because all letters need to be used when forming another word.
iiii oiii ooii oooi oooo is valid.
oiii ioii iioi iiio is not valid - any of these words can be rearranged to form any other word.
```
Under this new system policy, how many passphrases are valid?

In [ ]:
def no_anagrams(words):
    sorted_words = [''.join(s) for s in list(map(sorted, words))]
    return no_dupes(sorted_words)

assert no_anagrams(["cat", "act"]) == False
valid_passphrase_count(Input(4).read().splitlines(), no_anagrams)

# [Day 5](http://adventofcode.com/2017/day/5): A Maze of Twisty Trampolines, All Alike

### Part one
An urgent interrupt arrives from the CPU: it's trapped in a maze of jump instructions, and it would like assistance from any programs with spare cycles to help find the exit.

The message includes a list of the offsets for each jump. Jumps are relative: -1 moves to the previous instruction, and 2 skips the next one. Start at the first instruction in the list. The goal is to follow the jumps until one leads outside the list.

In addition, these instructions are a little strange; after each jump, the offset of that instruction increases by 1. So, if you come across an offset of 3, you would move three instructions forward, but change it to a 4 for the next time it is encountered.

For example, consider the following list of jump offsets:

```
0
3
0
1
-3
```
Positive jumps ("forward") move downward; negative jumps move upward. For legibility in this example, these offset values will be written all on one line, with the current instruction marked in parentheses. The following steps would be taken before an exit is found:

```
(0) 3  0  1  -3  - before we have taken any steps.
(1) 3  0  1  -3  - jump with offset 0 (that is, don't jump at all). Fortunately, the instruction is then incremented to 1.
 2 (3) 0  1  -3  - step forward because of the instruction we just modified. The first instruction is incremented again, now to 2.
 2  4  0  1 (-3) - jump all the way to the end; leave a 4 behind.
 2 (4) 0  1  -2  - go back to where we just were; increment -3 to -2.
 2  5  0  1  -2  - jump 4 steps forward, escaping the maze.
 ```
In this example, the exit is reached in 5 steps.

How many steps does it take to reach the exit?

In [ ]:
part_one = lambda x: x + 1

def trampoline(jumps, rule):
    loc = 0
    jump_count = 0
    while True:
        # index of next hop
        move = loc + jumps[loc]
        if move < 0 or move >= len(jumps):
            return jump_count + 1
        jumps[loc] = rule(jumps[loc])
        jump_count += 1
        loc = move

assert trampoline([0, 3, 0, 1, -3], part_one) == 5
trampoline([int(n) for n in Input(5).readlines()], part_one)

Now, the jumps are even stranger: after each jump, if the offset was three or more, instead decrease it by 1. Otherwise, increase it by 1 as before.

Using this rule with the above example, the process now takes 10 steps, and the offset values after finding the exit are left as 2 3 2 3 -1.

How many steps does it now take to reach the exit?

In [ ]:
part_two = lambda x: x - 1 if x > 2 else x + 1
assert trampoline([0, 3, 0, 1, -3], part_two) == 10
trampoline([int(n) for n in Input(5).readlines()], part_two)

# [Day 6](http://adventofcode.com/2017/day/6): Memory Reallocation

### Part one

A debugger program here is having an issue: it is trying to repair a memory reallocation routine, but it keeps getting stuck in an infinite loop.

In this area, there are sixteen memory banks; each memory bank can hold any number of blocks. The goal of the reallocation routine is to balance the blocks between the memory banks.

The reallocation routine operates in cycles. In each cycle, it finds the memory bank with the most blocks (ties won by the lowest-numbered memory bank) and redistributes those blocks among the banks. To do this, it removes all of the blocks from the selected bank, then moves to the next (by index) memory bank and inserts one of the blocks. It continues doing this until it runs out of blocks; if it reaches the last memory bank, it wraps around to the first one.

The debugger would like to know how many redistributions can be done before a blocks-in-banks configuration is produced that has been seen before.

For example, imagine a scenario with only four memory banks:

- The banks start with 0, 2, 7, and 0 blocks. 
- The third bank has the most blocks, so it is chosen for redistribution.
- Starting with the next bank (the fourth bank) and then continuing to the first bank, the second bank, and so on, the 7 blocks are spread out over the memory banks. The fourth, first, and second banks get two blocks each, and the third bank gets one back. The final result looks like this: 2 4 1 2.
- Next, the second bank is chosen because it contains the most blocks (four). Because there are four memory banks, each gets one block. The result is: 3 1 2 3.
- Now, there is a tie between the first and fourth memory banks, both of which have three blocks. The first bank wins the tie, and its three blocks are distributed evenly over the other three banks, leaving it with none: 0 2 3 4.
- The fourth bank is chosen, and its four blocks are distributed such that each of the four banks receives one: 1 3 4 1.
 -The third bank is chosen, and the same thing happens: 2 4 1 2.
 -At this point, we've reached a state we've seen before: 2 4 1 2 was already seen. The infinite loop is detected after the fifth block redistribution cycle, and so the answer in this example is 5.

Given the initial block counts in your puzzle input, how many redistribution cycles must be completed before a configuration is produced that has been seen before?

In [ ]:
INPUT = "0	5	10	0	11	14	13	4	11	8	8	7	1	4	12	11"

def redistribute(banks):
    block_count = max(banks)
    loc = banks.index(block_count)
    banks[loc] = 0
    
    while block_count > 0:
        loc = (loc + 1) % len(banks)
        banks[loc] += 1
        block_count -= 1

b = [0, 2, 7, 0]
redistribute(b)
assert b == [2, 4, 1, 2]

def find_loop(banks):
    seen_configs = set()
    cycles = 0
    
    while True:
        config = ".".join([str(n) for n in banks])
        if config in seen_configs:
            return cycles
        seen_configs.add(config)
        redistribute(banks)
        cycles += 1
    
assert find_loop([0, 2, 7, 0]) == 5

find_loop([int(n) for n in INPUT.split("	")])

### Part two

Out of curiosity, the debugger would also like to know the size of the loop: starting from a state that has already been seen, how many block redistribution cycles must be performed before that same state is seen again?

In the example above, 2 4 1 2 is seen again after four cycles, and so the answer in that example would be 4.

How many cycles are in the infinite loop that arises from the configuration in your puzzle input?

In [ ]:
def find_loop_size(banks):
    seen_configs = {}
    cycles = 0
    
    while True:
        config = ".".join([str(n) for n in banks])
        if config in seen_configs:
            return cycles - seen_configs[config]
        seen_configs[config] = cycles
        redistribute(banks)
        cycles += 1

assert find_loop_size([0, 2, 7, 0]) == 4
find_loop_size([int(n) for n in INPUT.split("	")])